In [1]:
import requests
import time
import urllib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
access_token='xxx' 
headers = {'Authorization':"Token "+access_token}   

### Define the functions : input - output
**created_at:** name of the repository - creation date of the repository \
**stargazers_count:** name of the repository - number of stars given to the repository \
**watchers_count:** name of the repository - number of watchers of the repository \
**open_issue_count:** name of the repository - # of open issues (not including pull requests) the repository \
**closed_issue_count:** name of the repository - # of closed issues (not including pull requests) the repository \
**open_pr_count:** name of the repository - # of open pull requests (not including other issues that are not pull requests) the repository \
**closed_pr_count:** name of the repository - # of closed pull requests (not including other issues that are not pull requests) the repository \
**commit_count:** name of the repository - # of commits made in the repository \
**release_count:** name of the repository - # of releases made in the repository \
**issue_info:** name of the repository - issue data as df \
**pullRequest_info:** name of the repository - pull request data as df \
**label_info:** name of the repository - label data of the labels that are used in the repository as df \
**release_info:** name of the repository - release data as df \
**issue_release_tag:** release_info,name of the repository,created_at - tag of the current release of the repository when the issue was created \
**issue_release_date:** release_info,name of the repository,created_at - date of the current release of the repository when the issue was created \
**issue_release_graph:** issue_info,name of the repository - graph of the number of issues created through the releases of the repository \


In [3]:
def created_at(repository):
    k = 0
    while k < 100:
        try:
            url = f"https://api.github.com/repos/{repository}"
            result = requests.get(url,headers=headers,verify=False).json()
            created_at = result['created_at']
            k = 100
        except:
            time.sleep(90)
            continue 
    return created_at

In [4]:
def stargazers_count(repository):
    k = 0
    while k < 100:
        try:
            url = f"https://api.github.com/repos/{repository}"
            result = requests.get(url,headers=headers,verify=False).json()
            stargazers_count = result['stargazers_count']
            k = 100
        except:
            time.sleep(90)
            continue 
    return stargazers_count

In [5]:
def watchers_count(repository):
    k = 0
    while k < 100:
        try:
            url = f"https://api.github.com/repos/{repository}"
            result = requests.get(url,headers=headers,verify=False).json()
            subscribers_count = result['subscribers_count']
            k = 100
        except:
            time.sleep(90)
            continue 
    return subscribers_count

In [6]:
def open_issue_count(repository):
    k = 0
    while k < 100:
        try:
            url = f"https://api.github.com/search/issues?q=repo:{repository}+type:issue+state:open"
            result = requests.get(url,headers=headers,verify=False).json()
            open_issue_count = result['total_count']
            k = 100
        except:
            time.sleep(90)
            continue 
    return open_issue_count

In [7]:
def closed_issue_count(repository):
    k = 0
    while k < 100:
        try:
            url = f"https://api.github.com/search/issues?q=repo:{repository}+type:issue+state:closed"
            result = requests.get(url,headers=headers,verify=False).json()
            closed_issue_count = result['total_count']
            k = 100
        except:
            time.sleep(90)
            continue 
    return closed_issue_count

In [8]:
def open_pr_count(repository):
    k = 0
    while k < 100:
        try:
            url = f"https://api.github.com/search/issues?q=repo:{repository}+type:pull-request+state:open"
            result = requests.get(url,headers=headers,verify=False).json()
            open_pr_count = result['total_count']
            k = 100
        except:
            time.sleep(90)
            continue 
    return open_pr_count

In [9]:
def closed_pr_count(repository):
    k = 0
    while k < 100:
        try:
            url = f"https://api.github.com/search/issues?q=repo:{repository}+type:pull-request+state:closed"
            result = requests.get(url,headers=headers,verify=False).json()
            closed_pr_count = result['total_count']
            k = 100
        except:
            time.sleep(90)
            continue 
    return closed_pr_count

In [10]:
def commit_count(repository):
    url = f'https://api.github.com/repos/{repository}/commits'
    headers = {'Authorization':"Token "+access_token, 'Accept': 'application/json', 'Content-Type': 'application/json'}
    params = {'per_page': 1}
    k = 0
    while k < 100:
        try:
            resp = requests.request('GET', url, params=params, headers=headers, verify=False)
            commit_count = len(resp.json())
            last_page = resp.links.get('last')
            # if there are no more pages, the count must be 0 or 1
            k = 1000
            if last_page:
                # extract the query string from the last page url
                qs = urllib.parse.urlparse(last_page['url']).query
                # extract the page number from the query string
                commit_count = int(dict(urllib.parse.parse_qsl(qs))['page'])
        except:
            time.sleep(90)
            continue
    return commit_count

In [11]:
def release_count(repository):
    url = f'https://api.github.com/repos/{repository}/releases'
    headers = {'Authorization':"Token "+access_token, 'Accept': 'application/json', 'Content-Type': 'application/json'}
    params = {'per_page': 1}
    k = 0
    while k < 100:
        try:
            resp = requests.request('GET', url, params=params, headers=headers, verify=False)
            release_count = len(resp.json())
            last_page = resp.links.get('last')
            # if there are no more pages, the count must be 0 or 1
            k = 1000
            if last_page:
                # extract the query string from the last page url
                qs = urllib.parse.urlparse(last_page['url']).query
                # extract the page number from the query string
                release_count = int(dict(urllib.parse.parse_qsl(qs))['page'])
        except:
            time.sleep(90)
            continue
    return release_count

In [12]:
def issue_info(repository):
    page_num = 1
    since = '1980-01-01'
    issues = []
    pg = 1
    while since and pg<1000:
        page_num = 1
        while page_num < 11:
            try:
                url = f"https://api.github.com/search/issues?q=is%3Aissue+created%3A>{since}+sort%3Acreated-asc+repo:{repository}&per_page=100&page={page_num}"
                issue = requests.get(url,headers=headers,verify=False).json()
                result = issue['items']
                if result == []:
                    page_num = 1000
                    since = None
                    pg = 10000
                else:  
                    issues.append(issue['items'])
                    page_num = page_num + 1
                    pg += 1
            except:
                time.sleep(120)
                continue
        since = issues[-1][-1]['created_at']
    url=[]
    repository_url=[]
    labels_url=[]
    comments_url=[]
    events_url=[]
    html_url=[]
    issue_id=[]
    node_id=[]
    number=[]
    title=[]
    user=[]
    labels=[]
    state=[]
    locked=[]
    assignee=[]
    assignees=[]
    milestone=[]
    comments=[]
    created_at=[]
    updated_at=[]
    closed_at=[]
    author_association=[]
    active_lock_reason=[]
    body=[]
    reactions=[]
    timeline_url=[]
    performed_via_github_app=[]
    state_reason=[]
    score=[]
    for page in issues:
        for issue in page:
            try:
                url.append(issue['url'])
                repository_url.append(issue['repository_url'])
                labels_url.append(issue['labels_url'])
                comments_url.append(issue['comments_url'])
                events_url.append(issue['events_url'])
                html_url.append(issue['html_url'])
                issue_id.append(issue['id'])
                node_id.append(issue['node_id'])
                number.append(issue['number'])
                title.append(issue['title'])
                user.append(issue['user'])
                labels.append(issue['labels'])
                state.append(issue['state'])
                locked.append(issue['locked'])
                assignee.append(issue['assignee'])
                assignees.append(issue['assignees'])
                milestone.append(issue['milestone'])
                comments.append(issue['comments'])
                created_at.append(issue['created_at'])
                updated_at.append(issue['updated_at'])
                closed_at.append(issue['closed_at'])
                author_association.append(issue['author_association'])
                active_lock_reason.append(issue['active_lock_reason'])
                body.append(issue['body'])
                reactions.append(issue['reactions'])
                timeline_url.append(issue['timeline_url'])
                performed_via_github_app.append(issue['performed_via_github_app'])
                state_reason.append(issue['state_reason'])
                score.append(issue['score'])
            except:
                pass
    issue_data=pd.DataFrame()
    issue_data['url']=url
    issue_data['repository_url']=repository_url
    issue_data['labels_url']=labels_url
    issue_data['comments_url']=comments_url
    issue_data['events_url']=events_url
    issue_data['html_url']=html_url
    issue_data['id']=issue_id
    issue_data['node_id']=node_id
    issue_data['number']=number
    issue_data['title']=title
    issue_data['user']=user
    issue_data['labels']=labels
    issue_data['state']=state
    issue_data['locked']=locked
    issue_data['assignee']=assignee
    issue_data['assignees']=assignees
    issue_data['milestone']=milestone
    issue_data['comments']=comments
    issue_data['created_at']=created_at
    issue_data['updated_at']=updated_at
    issue_data['closed_at']=closed_at
    issue_data['author_association']=author_association
    issue_data['active_lock_reason']=active_lock_reason
    issue_data['body']=body
    issue_data['reactions']=reactions
    issue_data['timeline_url']=timeline_url
    issue_data['performed_via_github_app']=performed_via_github_app
    issue_data['state_reason']=state_reason
    issue_data['score']=score
    issue_data=issue_data.reset_index().drop(columns='index')
    return issue_data

In [13]:
def pullRequest_info(repository):
    page_num = 1
    since = '1980-01-01'
    pullRequests = []
    pg = 1
    while since and pg<1000:
        page_num = 1
        while page_num < 11:
            try:
                url = f"https://api.github.com/search/issues?q=is%3Apull-request+created%3A>{since}+sort%3Acreated-asc+repo:{repository}&per_page=100&page={page_num}"
                pullRequest = requests.get(url,headers=headers,verify=False).json()
                result = pullRequest['items']
                if result == []:
                    page_num = 1000
                    since = None
                    pg = 10000
                else:  
                    pullRequests.append(pullRequest['items'])
                    page_num = page_num + 1
                    pg += 1
            except:
                time.sleep(90)
                continue
        since = pullRequests[-1][-1]['created_at']
    url=[]
    repository_url=[]
    labels_url=[]
    comments_url=[]
    events_url=[]
    html_url=[]
    pullRequest_id=[]
    node_id=[]
    number=[]
    title=[]
    user=[]
    labels=[]
    state=[]
    locked=[]
    assignee=[]
    assignees=[]
    milestone=[]
    comments=[]
    created_at=[]
    updated_at=[]
    closed_at=[]
    author_association=[]
    active_lock_reason=[]
    #draft=[]
    #pull_request=[]
    body=[]
    reactions=[]
    timeline_url=[]
    performed_via_github_app=[]
    state_reason=[]
    score=[]
    for page in pullRequests:
        for pullRequest in page:
            try:
                url.append(pullRequest['url'])
                repository_url.append(pullRequest['repository_url'])
                labels_url.append(pullRequest['labels_url'])
                comments_url.append(pullRequest['comments_url'])
                events_url.append(pullRequest['events_url'])
                html_url.append(pullRequest['html_url'])
                pullRequest_id.append(pullRequest['id'])
                node_id.append(pullRequest['node_id'])
                number.append(pullRequest['number'])
                title.append(pullRequest['title'])
                user.append(pullRequest['user'])
                labels.append(pullRequest['labels'])
                state.append(pullRequest['state'])
                locked.append(pullRequest['locked'])
                assignee.append(pullRequest['assignee'])
                assignees.append(pullRequest['assignees'])
                milestone.append(pullRequest['milestone'])
                comments.append(pullRequest['comments'])
                created_at.append(pullRequest['created_at'])
                updated_at.append(pullRequest['updated_at'])
                closed_at.append(pullRequest['closed_at'])
                author_association.append(pullRequest['author_association'])
                active_lock_reason.append(pullRequest['active_lock_reason'])
                #draft.append(pullRequest['draft'])
                #pull_request.append(pullRequest['pull_request'])
                body.append(pullRequest['body'])
                reactions.append(pullRequest['reactions'])
                timeline_url.append(pullRequest['timeline_url'])
                performed_via_github_app.append(pullRequest['performed_via_github_app'])
                state_reason.append(pullRequest['state_reason'])
                score.append(pullRequest['score'])
            except:
                pass
    pullRequest_data=pd.DataFrame()
    pullRequest_data['url']=url
    pullRequest_data['repository_url']=repository_url
    pullRequest_data['labels_url']=labels_url
    pullRequest_data['comments_url']=comments_url
    pullRequest_data['events_url']=events_url
    pullRequest_data['html_url']=html_url
    pullRequest_data['id']=pullRequest_id
    pullRequest_data['node_id']=node_id
    pullRequest_data['number']=number
    pullRequest_data['title']=title
    pullRequest_data['user']=user
    pullRequest_data['labels']=labels
    pullRequest_data['state']=state
    pullRequest_data['locked']=locked
    pullRequest_data['assignee']=assignee
    pullRequest_data['assignees']=assignees
    pullRequest_data['milestone']=milestone
    pullRequest_data['comments']=comments
    pullRequest_data['created_at']=created_at
    pullRequest_data['updated_at']=updated_at
    pullRequest_data['closed_at']=closed_at
    pullRequest_data['author_association']=author_association
    pullRequest_data['active_lock_reason']=active_lock_reason
    #pullRequest_data['draft']=draft
    #pullRequest_data['pull_request']=pull_request
    pullRequest_data['body']=body
    pullRequest_data['reactions']=reactions
    pullRequest_data['timeline_url']=timeline_url
    pullRequest_data['performed_via_github_app']=performed_via_github_app
    pullRequest_data['state_reason']=state_reason
    pullRequest_data['score']=score
    pullRequest_data=pullRequest_data.reset_index().drop(columns='index')
    return pullRequest_data    

In [14]:
def label_info(repository):
    page_num = 1
    labels = []
    while page_num < 11:
        try:
            url = f"https://api.github.com/repos/{repository}/labels?per_page=100&page={page_num}"
            label = requests.get(url,headers=headers,verify=False).json()
            result = label
            if result == []:
                page_num = 100
            else:
                labels.append(label)
                page_num = page_num + 1
        except:
            time.sleep(90)
            continue
    label_id = []
    label_node_id = []
    label_url = []
    label_name = []
    label_color = []
    label_default = []
    label_description = []
    for page in labels:
        for label in page:
            try:
                label_id.append(label['id'])
                label_node_id.append(label['node_id'])
                label_url.append(label['url'])
                label_name.append(label['name'])
                label_color.append(label['color'])
                label_default.append(label['default'])
                label_description.append(label['description'])
            except:
                pass
    label_data=pd.DataFrame()
    label_data['label_id']=label_id
    label_data['label_node_id']=label_node_id
    label_data['label_url']=label_url
    label_data['label_name']=label_name
    label_data['label_color']=label_color
    label_data['label_default']=label_default
    label_data['label_description']=label_description
    label_data=label_data.reset_index().drop(columns='index')
    return label_data

In [15]:
def release_info(repository):
    page_num = 1
    releases = []
    while page_num < 33:
        try:
            url = f"https://api.github.com/repos/{repository}/releases?page={page_num}"
            release = requests.get(url,headers=headers,verify=False).json()
            result = release
            if result == []:
                page_num = 100
            else:  
                releases.append(release)
                page_num = page_num + 1
        except:
            time.sleep(120)
            continue
    url=[]
    assets_url=[]
    upload_url=[]
    html_url=[]
    release_id=[]
    author=[]
    node_id=[]
    tag_name=[]
    target_commitish=[]
    name=[]
    draft=[]
    prerelease=[]
    created_at=[]
    published_at=[]
    assets=[]
    tarball_url=[]
    zipball_url=[]
    body=[]
    for page in releases:
        for release in page:
            try:
                url.append(release['url'])
                assets_url.append(release['assets_url'])
                upload_url.append(release['upload_url'])
                html_url.append(release['html_url'])
                release_id.append(release['id'])
                author.append(release['author'])
                node_id.append(release['node_id'])
                tag_name.append(release['tag_name'])
                target_commitish.append(release['target_commitish'])
                name.append(release['name'])
                draft.append(release['draft'])
                prerelease.append(release['prerelease'])
                created_at.append(release['created_at'])
                published_at.append(release['published_at'])
                assets.append(release['assets'])
                tarball_url.append(release['tarball_url'])
                zipball_url.append(release['zipball_url'])
                body.append(release['body'])
            except:
                pass
    release_data=pd.DataFrame()
    release_data['url']=url
    release_data['assets_url']=assets_url
    release_data['upload_url']=upload_url
    release_data['html_url']=html_url
    release_data['release_id']=release_id
    release_data['author']=author
    release_data['node_id']=node_id
    release_data['tag_name']=tag_name
    release_data['target_commitish']=target_commitish
    release_data['name']=name
    release_data['draft']=draft
    release_data['prerelease']=prerelease
    release_data['created_at']=created_at
    release_data['published_at']=published_at
    release_data['assets']=assets
    release_data['tarball_url']=tarball_url
    release_data['zipball_url']=zipball_url
    release_data['body']=body
    release_data=release_data.reset_index().drop(columns='index')
    return release_data

In [16]:
def issue_release_tag(releases,repository,created_at):
    release_df = releases[(releases['repository_name'] == repository)]
    release_df.sort_values(by=['published_at'], ascending = False, inplace = True)
    r = 0
    tag = 'prerelease'
    while r < release_df.shape[0]:
        if created_at < releases.loc[r,'published_at']:
            r += 1
        else:
            tag = releases.loc[r,'tag_name']
            break
    return tag

In [17]:
def issue_release_date(releases,repository,created_at):
    release_df = releases[releases['repository_name'] == repository]
    release_df.sort_values(by='published_at', ascending = False, inplace = True)
    r = 0
    release_date = '2010-01-01T00:00:00Z'
    while r < release_df.shape[0]:
        if created_at < releases.loc[r,'published_at']:
            r += 1
        else:
            release_date = releases.loc[r,'published_at']
            break
    return release_date

In [18]:
def issue_release_graph(allissues2,repository):
    allissues2_df = allissues2[allissues2['repository_name'] == repository]
    allissues2_df.sort_values(by='issue_release_published_at', inplace = True)
    sns.set_palette("PuOr_r")
    dim = (20,10)
    f, ax = plt.subplots(figsize=dim)
    sns.countplot(ax=ax, x="issue_release_tag_name", data=allissues2_df).set_title(repository)